In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 2.3 MB/s eta 0:00:00


In [2]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [3]:
!pip install ultralytics opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 23.8 MB/s eta 0:00:00


In [5]:
%%writefile app.py
import io
import time
import streamlit as st
import cv2
import numpy as np
import pandas as pd
import os
from ultralytics import YOLO
from io import BytesIO
from fpdf import FPDF
from datetime import datetime

# Dummy credentials for admin login
ADMIN_CREDENTIALS = {
    'admin': 'password123'  # Replace with actual admin credentials
}

# Function to check admin login
def check_login(username, password):
    return username in ADMIN_CREDENTIALS and ADMIN_CREDENTIALS[username] == password

# Function to update CSV
def update_csv(file_path, class_name, number_plate, detected, smoke_confidence, timestamp,bounding_box_area):
    try:
        if os.path.isfile(file_path):
            df = pd.read_csv(file_path)
        else:
            df = pd.DataFrame(columns=['Class', 'Number Plate', 'Count', 'Smoke', 'Smoke Confidence', 'Timestamp','intensity%'])

        existing_row = df[(df['Class'] == class_name) & (df['Number Plate'] == number_plate)]
        if existing_row.empty:
            new_row = pd.DataFrame([[class_name, number_plate, 1, detected, smoke_confidence, timestamp,smoke_confidence*100]],
                                   columns=['Class', 'Number Plate', 'Count', 'Smoke', 'Smoke Confidence', 'Timestamp','intensity%'])
            df = pd.concat([df, new_row], ignore_index=True)
        else:
            df.loc[existing_row.index, 'Count'] += 1
            df.loc[existing_row.index, 'Smoke Confidence'] = smoke_confidence
            df.loc[existing_row.index, 'Timestamp'] = timestamp
            df.loc[existing_row.index, 'intensity%'] = smoke_confidence*100
        df.to_csv(file_path, index=False)
    except Exception as e:
        st.error(f"⚠️ Error: {e}")

# Function to download CSV
def download_csv(file_path):
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        csv_buffer = BytesIO()
        df.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)
        st.download_button(
            label="Download CSV",
            data=csv_buffer,
            file_name="output.csv",
            mime="text/csv",
            key="download_csv"
        )

# Function to generate challan
def generate_challan(number_plate, class_name, smoke_confidence, fine_amount):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    pdf.cell(200, 10, txt="Traffic Violation Challan", ln=True, align='C')
    pdf.ln(10)

    pdf.cell(200, 10, txt=f"Vehicle Class: {class_name}", ln=True)
    pdf.cell(200, 10, txt=f"Number Plate: {number_plate}", ln=True)
    pdf.cell(200, 10, txt=f"Violation: Smoke Detected", ln=True)
    pdf.cell(200, 10, txt=f"Smoke Detection Confidence: {smoke_confidence:.2f}", ln=True)
    pdf.cell(200, 10, txt=f"Fine Amount: Rs {fine_amount}", ln=True)
    pdf.cell(200, 10, txt=f"Date and Time: {current_datetime}", ln=True)
    pdf.ln(10)
    pdf.cell(200, 10, txt="Please adhere to traffic rules and regulations.", ln=True)

    challan_path = f"challan_{number_plate}.pdf"
    pdf.output(challan_path)
    return challan_path

# Function to process each frame or image
# Function to process each frame or image
def process_frame(frame, vehicle_model, plate_model, char_model, smoke_model, smoke_threshold):
    results = vehicle_model(frame)

    for result in results:
        boxes = result.boxes.xyxy.tolist()
        classes = result.boxes.cls.tolist()
        names = result.names
        confidences = result.boxes.conf.tolist()

        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = map(int, box)
            classname = names[int(cls)]

            # Vehicle ROI
            vehicle_roi = frame[y1:y2, x1:x2]

            # License plate detection
            plate_results = plate_model(vehicle_roi)
            plate_str = ""
            s = ''
            for plate_result in plate_results:
                if len(plate_result.boxes.xyxy) > 0:
                    plate_box = plate_result.boxes.xyxy.tolist()[0]
                    x1p, y1p, x2p, y2p = map(int, plate_box)

                    # License plate ROI
                    plate_roi = vehicle_roi[y1p:y2p, x1p:x2p]
                    cv2.rectangle(frame, (x1 + x1p, y1 + y1p), (x1 + x2p, y1 + y2p), (0, 255, 255), 2)
                    cv2.putText(frame, 'License Plate', (x1 + x1p, y1 + y1p - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

                    char_results = char_model(plate_roi)
                    detected_objects = [{'class': char_result.names[int(cls)], 'x1': int(box[0]), 'y1': int(box[1])}
                                        for char_result in char_results
                                        for box, cls in zip(char_result.boxes.xyxy.tolist(), char_result.boxes.cls.tolist())]

                    sorted_objects = sorted(detected_objects, key=lambda x: (x['y1'], x['x1']))
                    if sorted_objects:
                        current = sorted_objects[0]['y1']
                        q, w, u, d = [], [], [], []
                        for i in sorted_objects:
                            if current - 3 <= i['y1'] <= current + 3:
                                q.append(i['class'])
                                w.append(i['x1'])
                            else:
                                u.append(i['class'])
                                d.append(i['x1'])

                        # Sort characters by x1 position
                        q = [char for _, char in sorted(zip(w, q))]
                        u = [char for _, char in sorted(zip(d, u))]

                        s = ''.join(q + u)

                    plate_str = s
                    cv2.putText(frame, f'Plate: {s}', (x1 + x1p, y1 + y2p + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

            # Smoke detection (only considering the "smoke" class and confidence < smoke_threshold)
            y1_adjusted = max(y1 + 70, 0)
            y2_adjusted = min(y2 + 70, frame.shape[0])
            x1_adjusted = max(x1 - 50, 0)
            x2_adjusted = min(x2 + 50, frame.shape[1])

            vehicle_roi1 = frame[y1_adjusted:y2_adjusted, x1_adjusted:x2_adjusted]
            smoke_results = smoke_model(vehicle_roi1)
            smoke_detected = False
            for smoke_result in smoke_results:
                smoke_boxes = smoke_result.boxes.xyxy.tolist()
                smoke_classes = smoke_result.boxes.cls.tolist()
                smoke_confs = smoke_result.boxes.conf.tolist()

                for smoke_box, smoke_cls, smoke_conf in zip(smoke_boxes, smoke_classes, smoke_confs):
                    if smoke_conf < smoke_threshold:  # Only consider smoke class and confidence < threshold
                        x1s, y1s, x2s, y2s = map(int, smoke_box)
                        bounding_box_area = (x2s - x1s) * (y2s - y1s)
                        cv2.rectangle(frame, (x1_adjusted + x1s, y1_adjusted + y1s), (x1_adjusted + x2s, y1_adjusted + y2s), (0, 255, 0), 2)
                        cv2.putText(frame, 'Smoke Detected', (x1_adjusted + x1s, y1_adjusted + y1s - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                        smoke_detected = True

            if len(s) > 8 and smoke_detected:  # Ensure valid number plate length and smoke detected
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                detected = 'Yes'
                update_csv('output.csv', classname, s, detected, smoke_conf, timestamp,bounding_box_area)

                fine_amount = 100  # Example fine amount, can be dynamic
                challan_path = generate_challan(s, classname, smoke_conf, fine_amount)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
            cv2.rectangle(frame, (x1_adjusted, y1_adjusted), (x2_adjusted, y2_adjusted), (255, 0, 255), 2)
            cv2.putText(frame, classname, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)

    return frame


# Streamlit login form
def login():
    st.title("🔐 Admin Login")
    st.subheader("Please enter your credentials to login.")

    username = st.text_input("👤 Username")
    password = st.text_input("🔑 Password", type='password')

    if st.button("Login"):
        if check_login(username, password):
            st.session_state['logged_in'] = True
            st.success("🎉 Logged in successfully!")
        else:
            st.error("❌ Invalid username or password")

# Main application logic
# Main application logic
def main_app():
    st.title("🚗 Smoke and Vehicle Detection and Processing App")

    st.sidebar.title("Upload Options")
    uploaded_file = st.sidebar.file_uploader("📁 Choose a video or image file", type=["mp4", "avi", "mov", "png", "jpg", "jpeg"])

    # Smoke detection threshold slider
    smoke_threshold = st.sidebar.slider("🟢 Smoke Detection Confidence Threshold", 0.0, 1.0, 0.5, 0.05)

    if uploaded_file:
        input_path = f"temp_{uploaded_file.name}"

        with open(input_path, 'wb') as f:
            f.write(uploaded_file.read())

        if input_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            st.image(input_path, caption='📷 Uploaded Image', use_column_width=True)

            vehicle_model = YOLO("/content/Veh.pt")
            plate_model = YOLO("/content/num.pt")
            char_model = YOLO("/content/char.pt")
            smoke_model = YOLO("/content/smoke.pt")

            image = cv2.imread(input_path)
            processed_image = process_frame(image, vehicle_model, plate_model, char_model, smoke_model, smoke_threshold)
            st.success("✅ CSV file updated successfully!")
            output_image_path = 'processed_image.jpg'
            cv2.imwrite(output_image_path, processed_image)
            st.image(processed_image, caption='✅ Processed Image', use_column_width=True)

            with open(output_image_path, "rb") as file:
                st.download_button(label="💾 Download Processed Image", data=file, file_name="processed_image.jpg", mime="image/jpeg", key="download_image")

        else:
            st.video(input_path)

            vehicle_model = YOLO("/content/Veh.pt")
            plate_model = YOLO("/content/num.pt")
            char_model = YOLO("/content/char.pt")
            smoke_model = YOLO("/content/smoke.pt")

            video_capture = cv2.VideoCapture(input_path)

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out_fps = video_capture.get(cv2.CAP_PROP_FPS)
            frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

            output_video_path = 'output_video.mp4'
            video_writer = cv2.VideoWriter(output_video_path, fourcc, out_fps, (frame_width, frame_height))

            while video_capture.isOpened():
                ret, frame = video_capture.read()
                if not ret:
                    break

                processed_frame = process_frame(frame, vehicle_model, plate_model, char_model, smoke_model, smoke_threshold)
                video_writer.write(processed_frame)

            video_capture.release()
            video_writer.release()

            st.video(output_video_path)
            st.success("✅ CSV file updated successfully!")

            with open(output_video_path, "rb") as file:
                st.download_button(label="💾 Download Processed Video", data=file, file_name="processed_video.mp4", mime="video/mp4", key="download_video")

        # Add CSV download button
        download_csv('output.csv')

        # Find row with maximum intensity percentage
        try:
            df = pd.read_csv('output.csv')
            max_intensity_row = df.loc[df['intensity%'].idxmax()]
            max_intensity = max_intensity_row['intensity%']
            number_plate = max_intensity_row['Number Plate']
            class_name = max_intensity_row['Class']
            smoke_confidence = max_intensity_row['Smoke Confidence']

            st.write(f"📊 Maximum intensity percentage is {max_intensity:.2f}% for vehicle with number plate: {number_plate}.")

            # Generate challan for the row with maximum intensity
            fine_amount = 100  # Example fine amount, can be dynamic
            challan_path = generate_challan(number_plate, class_name, smoke_confidence, fine_amount)
            with open(challan_path, "rb") as file:
                st.download_button(label="💾 Download Challan", data=file, file_name=challan_path, mime="application/pdf", key=f"challan_{number_plate}_{time.time()}")

        except Exception as e:
            st.error(f"⚠️ Error finding maximum intensity row: {e}")


# Main execution flow
if 'logged_in' not in st.session_state:
    st.session_state['logged_in'] = False

if not st.session_state['logged_in']:
    login()
else:
    main_app()


Writing app.py


In [4]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=a0bb3c34f034fcc50856e32f1e02f33dd10b041e0cd6cd65766f54c9936f53bd
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [7]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.145.91.97
your url is: https://clean-comics-suffer.loca.lt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
